In [21]:
import os
import re
import json
from pathlib import Path
from copy import deepcopy

In [83]:
BASE_ANNOTATION_DIR = Path("/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Data/Webanno/robot_commands_2022-12-13_2126/annotation/")

In [94]:
projects = {}
for projectDirName in ["1 Движение на заданное количество метров в определенном направлении.txt",
                        "2 Поворот на заданное количество градусов в определенном направлении.txt",
                        "3 Команда «разворот».txt",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3.txt",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота.txt",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда.txt",
                        "11 Патрулирование по заданному маршруту.txt",
                        "12 Патрулирование по кругу с заданным радиусом.txt",
                        "13 Команда следования за объектом.txt"]:
    
    parsed_commands = {}
    for userFileName in ["student1.tsv", "student2.tsv", "PharmExpert_1.tsv"]:
        if not os.path.exists(BASE_ANNOTATION_DIR / projectDirName / userFileName):
            print("no", userFileName, "for", projectDirName)
            continue
        with open(BASE_ANNOTATION_DIR / projectDirName / userFileName, "r", encoding="utf-8") as f:
            parsed_commands[userFileName] = []
            skiping_attributes_line = False
            current_command = None
            for line in f:
                line = line.strip()
                if line=="":
                    continue
                if line[:6] == "#Text=":
                    line = line.encode().decode('unicode_escape')
                    if "action=" in line:
                        # save old 
                        if current_command is not None:
                            assert current_command["text"] is not None
                            parsed_commands[userFileName].append(current_command)
                        # begin new doc
                        current_command = {
                            "attributes_text": re.sub("#Text=", "", line),
                            "text": None,
                            "parse": []
                        }
                        skiping_attributes_line = True
                    else:
                        assert current_command["text"] is None
                        current_command["text"] = re.sub("#Text=", "", line)
                        skiping_attributes_line = False
                elif line[0]=="#":
                    continue
                else:
                    if skiping_attributes_line:
                        continue
                    line = line.split("\t")
                    if len(line)==4:
                        if line[3]=="*":
                            print(projectDirName, word)
                            raise ValueError("Удали звёздочки в файлах, это косяки разметки")
                        # удаляем индексы сущностей, они нам вроде как не нужны 
                        line[3] = re.sub("\\[\d+\\]", "", line[3])
                    else:
                        line.append("_")
                    #current_command["parse"].append((line[2], line[3] if len(line)==4 else "_"))
                    current_command["parse"].append({"form":line[2],
                                                    "attribute_name":line[3]})
                    
            parsed_commands[userFileName].append(current_command)
    projects[projectDirName] = parsed_commands

no student1.tsv for 1 Движение на заданное количество метров в определенном направлении.txt
no student2.tsv for 1 Движение на заданное количество метров в определенном направлении.txt
no student1.tsv for 2 Поворот на заданное количество градусов в определенном направлении.txt
no student2.tsv for 2 Поворот на заданное количество градусов в определенном направлении.txt
no student1.tsv for 3 Команда «разворот».txt
no student2.tsv for 3 Команда «разворот».txt
no student1.tsv for 5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt
no student2.tsv for 5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt
no student1.tsv for 6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt
no student2.tsv for 6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt
no student1.tsv for 7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt
no student2.tsv for 7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt
no student1.tsv for 8 По

In [95]:
# проверка на то, что всегда есть только один разметчик
for projectDirName in ["1 Движение на заданное количество метров в определенном направлении.txt",
                        "2 Поворот на заданное количество градусов в определенном направлении.txt",
                        "3 Команда «разворот».txt",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3.txt",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота.txt",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда.txt",
                        "11 Патрулирование по заданному маршруту.txt",
                        "12 Патрулирование по кругу с заданным радиусом.txt",
                        "13 Команда следования за объектом.txt"]:
    parsed_commands = projects[projectDirName]
    assert 'PharmExpert_1.tsv' in parsed_commands
    assert 'student1.tsv' not in parsed_commands
    assert 'student2.tsv' not in parsed_commands

In [96]:
# оставить только одного разметчика
from collections import Counter
users_counter = Counter()
for projectDirName in ["1 Движение на заданное количество метров в определенном направлении.txt",
                        "2 Поворот на заданное количество градусов в определенном направлении.txt",
                        "3 Команда «разворот».txt",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3.txt",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота.txt",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда.txt",
                        "11 Патрулирование по заданному маршруту.txt",
                        "12 Патрулирование по кругу с заданным радиусом.txt",
                        "13 Команда следования за объектом.txt"]:
    parsed_commands = projects[projectDirName]
    new_list = []
    for i in range(len(parsed_commands['PharmExpert_1.tsv'])):        
        users_counter["PharmExpert_1"] += 1
        parse = parsed_commands['PharmExpert_1.tsv'][i]
        simplified_command = deepcopy(parsed_commands['PharmExpert_1.tsv'][i])
        #simplified_command["parse"] = parse["parse"][2:]
        simplified_command["parse"] = parse["parse"]  # !!
        new_list.append(simplified_command)
    projects[projectDirName] = new_list

In [97]:
# проверить, нет ли каких-то корявых лейблов
labels_set = set()
for projectDirName in ["1 Движение на заданное количество метров в определенном направлении.txt",
                        "2 Поворот на заданное количество градусов в определенном направлении.txt",
                        "3 Команда «разворот».txt",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3.txt",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота.txt",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда.txt",
                        "11 Патрулирование по заданному маршруту.txt",
                        "12 Патрулирование по кругу с заданным радиусом.txt",
                        "13 Команда следования за объектом.txt"]:
    parsed_commands = projects[projectDirName]
    for command in parsed_commands:
        for word in command["parse"]:
            labels_set.add(word["attribute_name"])
labels_set

{'_',
 'action',
 'action|degshours',
 'action|direction',
 'degshours',
 'direction',
 'direction|degshours',
 'gaze',
 'meters',
 'nearest',
 'object1',
 'object1|object2',
 'object1|relation1',
 'object2',
 'object3',
 'relation1',
 'relation2',
 'self'}

In [98]:
descriptions = {
    "action": ["патрулировать", "остановиться", "двигаться в направлении", "поворот в направлениие",
               "двигаться на заданное расстояние", "поворачивать на заданный угол", "двигаться к объекту",
               "искать", "обходить", "осматривать", "поворачивать к объекту", "прерваться", "продолжать", "анализ"],
    "direction": ["", "вперед", "назад", "направо", "налево", "на север", "на юг", "на восток", "на запад"],
    "meters": ["", "1", "3", "5", "7", "10", "15", "20", "25", "35", "50", "75","100"],
    "degshours": ["", "15", "30", "45", "60", "75","90", "105", "120", "135", "150","165", "180", "195", "210","225", "240", "255", "270","285", "300", "315", "330","345", "360"],
    "object1": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "nearest": ["",  "nearest_from_type"],
    "relation1": ["нет_отношения", "около", "позади", "перед","слева от", "правее"],
    "object2": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "relation2": ["нет_отношения", "около", "позади", "перед","слева от", "правее"],
    "object3": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "self": ["", "behind_of", "in_front_of", "to_left_from","to_right_from"],
    "gaze": ["", "has_gaze_focus_on"]
}
descriptions = {
    "y": ["simple", "direction", "meter", "degree", "hour", "object", "nearest", "relation1", "relation2", "circle", "self", "gaze"],
    "action": ["patrol", "stop", "move_dir", "rotate_dir", "move_on", "rotate_on", "move_to", "find", "go_around", "monitor", "rotate_to", "pause", "continue", "analyze", "follow"],
    "direction": ["", "dir_forward", "dir_backward", "dir_right", "dir_left", "dir_north", "dir_south", "dir_east", "dir_west"],
    "meters": ["", "1", "3", "5", "7", "10", "15", "20", "25", "35", "50", "75","100"],
    "degshours": ["", "15", "30", "45", "60", "75","90", "105", "120", "135", "150","165", "180", "195", "210","225", "240", "255", "270","285", "300", "315", "330","345", "360"],
    "object1": ["", "house", "tree", "broken_tree","forest", "pit", "human", "hill","fissure", "man_lay", "rock", "butte","barrier", "lamp_post", "car", "circle", "route"],
    "nearest": ["",  "nearest_from_type"],
    "relation1": ["", "near", "behind_of", "in_front_of","to_left_from", "to_right_from", "to_north_from", "to_south_from","to_east_from", "to_west_from"],
    "object2": ["", "house", "tree", "broken_tree","forest", "pit", "human", "hill","fissure", "man_lay", "rock", "butte","barrier", "lamp_post", "car"],
    "relation2": ["", "near", "behind_of", "in_front_of","to_left_from", "to_right_from", "to_north_from", "to_south_from","to_east_from", "to_west_from"],
    "object3": ["", "house", "tree", "broken_tree","forest", "pit", "human", "hill","fissure", "man_lay", "rock", "butte","barrier", "lamp_post", "car"],
    "self": ["", "behind_of", "in_front_of", "to_left_from","to_right_from", "to_north_from", "to_south_from","to_east_from", "to_west_from"],
    "gaze": ["", "has_gaze_focus_on"]
}
webannolabel_to_shortlabel = {
    '_': ["O"],
     'action': ["A"],
     'direction': ["D"],
     'object1': ["O1"],
     'object2': ["O2"],
     'object3': ["O3"],
     'relation1': ["R1"],
     'relation2': ["R2"],
    'object1|relation1': ['O1', "R1"],
    'object1|object2': ['O1', "O2"],
    'action|degshours': ['A', "DH"],
    'action|direction': ['A', "D"],
    'degshours|action': ['A', "DH"],
    'direction|degshours': ['D', "DH"],
    "meters": ["M"],
    "degshours": ["DH"],
    "nearest": ["N"],
    "self":["S"],
    "gaze": ["G"]
}
for projectDirName in ["1 Движение на заданное количество метров в определенном направлении.txt",
                        "2 Поворот на заданное количество градусов в определенном направлении.txt",
                        "3 Команда «разворот».txt",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту.txt",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту.txt",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2.txt",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3.txt",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота.txt",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда.txt",
                        "11 Патрулирование по заданному маршруту.txt",
                        "12 Патрулирование по кругу с заданным радиусом.txt",
                        "13 Команда следования за объектом.txt"]:
    parsed_commands = projects[projectDirName]
    for command in parsed_commands:
        attributes_vector = [0,0,0,0,0,0,0,0,0,0,0,0]
        if command["attributes_text"][-1]==";":
            command["attributes_text"] = command["attributes_text"][:-1]
        command["attributes_text"] = command["attributes_text"].replace("action=rotate_to gaze=by_gaze_line;", "action=rotate_to; gaze=by_gaze_line;")
        for attr_pair in command["attributes_text"].split(";"):
            attr_name, attr_value = attr_pair.strip().split("=")
            attr_name = attr_name.replace("objsect1", "object1")
            if attr_name=="action":
                attributes_vector[0] = descriptions["action"].index(attr_value)
            elif attr_name=="object1":
                attr_value = attr_value.replace("hills", "hill")
                attr_value = attr_value.replace("carы", "car")
                attr_value = attr_value.replace("man", "human")
                attr_value = attr_value.replace("huhuman", "human")
                attr_value = attr_value.replace("human_lay", "man_lay")
                attributes_vector[4] = descriptions["object1"].index(attr_value)
            elif attr_name=="object2":
                attributes_vector[7] = descriptions["object2"].index(attr_value)
            elif attr_name=="object3":
                attr_value = attr_value.replace("fissures", "fissure")
                attributes_vector[9] = descriptions["object3"].index(attr_value)
            elif attr_name=="relation1":
                attr_value = attr_value.replace("to_near", "near")
                attributes_vector[6] = descriptions["relation1"].index(attr_value)
            elif attr_name=="relation2":
                attr_value = attr_value.replace("to_near", "near")
                if attr_value=="house":
                    print(projectDirName, "CHANGE RELATION=house to relation=near", command)
                    attr_value = "near"
                attributes_vector[8] = descriptions["relation2"].index(attr_value)            
            elif attr_name=="direction":
                attributes_vector[1] = descriptions["direction"].index(attr_value)
            elif attr_name=="meters":
                if attr_value in descriptions["meters"]:
                    attributes_vector[2] = descriptions["meters"].index(attr_value)
                else:
                    closest = min(descriptions["meters"][1:], key=lambda x:abs(int(x)-int(attr_value)))
                    attributes_vector[2] = descriptions["meters"].index(closest)
            elif attr_name=="degshours":
                if attr_value in descriptions["degshours"]:
                    attributes_vector[3] = descriptions["degshours"].index(attr_value)
                else:
                    closest = min(descriptions["degshours"][1:], key=lambda x:abs(int(x)-int(attr_value)))
                    attributes_vector[3] = descriptions["degshours"].index(closest)
            elif attr_name=="nearest":
                attributes_vector[5] = descriptions["nearest"].index(attr_value)
            elif attr_name=="self":
                attributes_vector[10] = descriptions["self"].index(attr_value)
            elif attr_name=="gaze":
                attr_value = attr_value.replace("by_gaze_line", "has_gaze_focus_on")
                attributes_vector[11] = descriptions["gaze"].index(attr_value)
            else:
                raise ValueError("Хз, что за атрибут")
        command["attributes_vector"] = attributes_vector
        for word in command["parse"]:
            word["attribute_name"] = webannolabel_to_shortlabel[word["attribute_name"]]
    with open("ForWebannoProject/annotated/"+projectDirName.replace(".txt", ".jsonl"), "w") as f:
        for command in parsed_commands:
            f.write(json.dumps(command)+"\n")

In [99]:
from collections import Counter
labels = ['A', 'D', 'M', 'DH', 'O1', 'N', 'R1', 'O2', 'R2', 'O3', 'S', 'G',]
# проверка, как много команд, где нет нёров для атрибутов команды, и все ли они такие, что в них нельзя выделить
for projectName in ["1 Движение на заданное количество метров в определенном направлении",
                        "2 Поворот на заданное количество градусов в определенном направлении",
                        "3 Команда «разворот»",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда",
                        "11 Патрулирование по заданному маршруту",
                        "12 Патрулирование по кругу с заданным радиусом",
                        "13 Команда следования за объектом"]:
    print(projectName)
    with open(f"/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject/annotated/{projectName}.jsonl", "r") as f:
        noWordsForAttrCounter = Counter()
        totalCounter = Counter()
        totalCounter_2 = Counter()
        for line in f:
            data = json.loads(line)
            attrvec = set()
            for i in range(len(data["attributes_vector"])):
                if data["attributes_vector"][i]!=0:
                    totalCounter[labels[i]]+=1
                    attrvec.add(labels[i])
                elif labels[i]=="A":
                    attrvec.add(labels[i])
            attrwords =set()
            for word in data["parse"]:
                attrwords.update(word["attribute_name"])
            for l in attrwords:
                totalCounter_2[l]+=1
            attrwords = attrwords - set(["O"])
            if len(attrwords - attrvec) !=0 :
                print("not equal sets")
                print(attrwords, attrvec)
                print(data)
            for x in attrvec-attrwords:
                noWordsForAttrCounter[x]+=1
    print(totalCounter)
    print(totalCounter_2)
    print(noWordsForAttrCounter)
    print()

1 Движение на заданное количество метров в определенном направлении
Counter({'A': 51, 'D': 51, 'M': 51})
Counter({'M': 51, 'A': 48, 'D': 47, 'O': 45})
Counter({'D': 4, 'A': 3})

2 Поворот на заданное количество градусов в определенном направлении
Counter({'A': 50, 'D': 50, 'DH': 50})
Counter({'O': 50, 'A': 50, 'DH': 50, 'D': 25})
Counter({'D': 25})

3 Команда «разворот»
Counter({'A': 45, 'D': 45, 'DH': 45})
Counter({'DH': 45, 'A': 39, 'O': 28, 'D': 1})
Counter({'D': 44, 'A': 6})

5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту
Counter({'A': 50, 'O1': 50, 'N': 50})
Counter({'O1': 50, 'A': 50, 'N': 50, 'O': 48})
Counter()

6 Поиск_объезд_осмотр_анализ_поворот к объекту
Counter({'A': 50, 'O1': 50})
Counter({'A': 50, 'O1': 50, 'O': 47})
Counter()

7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2
Counter({'A': 50, 'O1': 50, 'R1': 50, 'O2': 50})
Counter({'R1': 50, 'A': 50, 'O1': 50, 'O2': 50, 'O': 47})
Counter()

8 Поиск_объезд_осмотр_анализ_поворот 

- разделить колонку degshours на degs и hours
- переразметить hours

In [177]:
for projectName in ["1 Движение на заданное количество метров в определенном направлении",
                        "2 Поворот на заданное количество градусов в определенном направлении",
                        "3 Команда «разворот»",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда",
                        "11 Патрулирование по заданному маршруту",
                        "12 Патрулирование по кругу с заданным радиусом",
                        "13 Команда следования за объектом"]:
    print(projectName)
    with open(f"/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject/annotated/{projectName}.jsonl", "r") as f:
        newlines = []
        for line in f:
            data = json.loads(line)
            # во всех проектах кроме двух аттрибут degshours должен быть нулевым
            if projectName not in ["2 Поворот на заданное количество градусов в определенном направлении",
                                  "3 Команда «разворот»"]:
                assert data["attributes_vector"][3] == 0
                assert "DH" not in [x for sublist in data["parse"] for x in sublist["attribute_name"]]
                assert "DS" not in [x for sublist in data["parse"] for x in sublist["attribute_name"]]
                assert "HS" not in [x for sublist in data["parse"] for x in sublist["attribute_name"]]
                # в этом случае просто добавляем атрибут hours с нулём и ничего не меняется больше.
                data["attributes_vector"].insert(4,0)
            elif projectName in ["2 Поворот на заданное количество градусов в определенном направлении"]:
                # а в двух проектах в которых есть повороты есть нюанс
                if "град" not in data["text"]:
                    data["attributes_vector"].insert(3,0)
                    print("HOURS\t", data["text"])
                    for w_i in range(len(data["parse"])):
                        for m_i in range(len(data["parse"][w_i]["attribute_name"])):
                            if data["parse"][w_i]["attribute_name"][m_i]=="DH":
                                data["parse"][w_i]["attribute_name"][m_i] = "HS"
                else:
                    data["attributes_vector"].insert(4,0)
                    for w_i in range(len(data["parse"])):
                        for m_i in range(len(data["parse"][w_i]["attribute_name"])):
                            if data["parse"][w_i]["attribute_name"][m_i]=="DH":
                                data["parse"][w_i]["attribute_name"][m_i] = "DS"
                    print("GRAD\t", data["text"])
            elif projectName in ["3 Команда «разворот»"]:
                data["attributes_vector"].insert(4,0)
                assert data["attributes_vector"][3]==12 
                assert data["attributes_vector"][0]==5 
                assert data["attributes_vector"][1]==4
            data.pop("attributes_text")            
            newlines.append(json.dumps(data))
    with open(f"/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject/annotated_fixed/{projectName}.jsonl", "w") as f:
        for line in newlines:
            f.write(line+"\n")


1 Движение на заданное количество метров в определенном направлении
2 Поворот на заданное количество градусов в определенном направлении
GRAD	 поверни по левую руку на 180 градусов
GRAD	 поворачивай-ка влево на 45 градусов
HOURS	 сделай поворот на 3 часа
HOURS	 поверни на 6 часов
HOURS	 соверши поворот через на 8 часов
HOURS	 сделай поворот на 9 часов
GRAD	 поворачивай против часовой стрелки на 37 градусов
GRAD	 заверни в левую сторону на 60 градусов
HOURS	 нужно повернуть на 6 часов
HOURS	 заворачивай-ка на 4 часа
HOURS	 соверши поворот на 2 часа
HOURS	 на 5 часов поверни
GRAD	 крутанись-ка вправо на 100 градусов
GRAD	 необходимо развернуться на 150 градусов через правое плечо
GRAD	 вращайся на 120 градусов по часовой стрелке
GRAD	 оглядись на 45 градусов через правое плечо
GRAD	 повернись через юг на 30 градусов
GRAD	 на 20 градусов заверни в южную сторону
GRAD	 развернись на 70 градусов южнее
GRAD	 тебе нужно завернуть на 35 градусов на юг
HOURS	 соверши поворот на 4 часа
HOURS	 зав

In [176]:
data["parse"]

[{'form': 'ринься', 'attribute_name': ['A']},
 {'form': 'за', 'attribute_name': ['A']},
 {'form': 'машинкой', 'attribute_name': ['O1']},
 {'form': '!', 'attribute_name': ['O']}]

In [178]:
total_lines = []
for projectName in ["1 Движение на заданное количество метров в определенном направлении",
                        "2 Поворот на заданное количество градусов в определенном направлении",
                        "3 Команда «разворот»",
                        "5 Движение_поиск_объезд_осмотр_анализ_поворот к ближайшему объекту",
                        "6 Поиск_объезд_осмотр_анализ_поворот к объекту",
                        "7 Движение_поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2",
                        "8 Поиск_объезд_осмотр_анализ_поворот к объекту1 около объекта2 рядом с объектом3",
                        "9_Движение_поиск_объезд_осмотр_анализ_поворот_к_объекту_относительно_робота",
                        "10 Движение_объезд_осмотр_анализ_поворот к объекту по направлению взгляда",
                        "11 Патрулирование по заданному маршруту",
                        "12 Патрулирование по кругу с заданным радиусом",
                        "13 Команда следования за объектом"]:
    
    with open(f"/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject/annotated_fixed/{projectName}.jsonl", "r") as f:
        for line in f:
            total_lines.append(json.loads(line))


In [168]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
for fold_i, (train_index, test_index) in enumerate(skf.split(list(range(len(total_lines))), [x['attributes_vector'][0] for x in total_lines])):
    for index in test_index:
        total_lines[index]["fold"] = fold_i

In [169]:
with open("/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Data/Raw/NERannontated_crowd_2.jsonl", "w") as f:
    for line in total_lines:
        f.write(json.dumps(line)+"\n")